## 准备数据


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2025-03-23 18:08:34.800218: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 18:08:34.820806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742724514.843047  812299 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742724514.849733  812299 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 18:08:34.874541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Demo numpy based auto differentiation


In [2]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        '''计算矩阵乘法的对应的梯度'''
        ####################
        grad_x = np.matmul(grad_y, W.T)
        grad_W = np.matmul(x.T, grad_y)
        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        ####################
        grad_x = np.where(self.mem['x'] > 0, grad_y, np.zeros_like(grad_y))
        return grad_x
    


class Softmax:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check


In [4]:
import tensorflow as tf

x = np.random.normal(size=[5, 6])
W = np.random.normal(size=[6, 4])
aa = Matmul()
out = aa.forward(x, W) # shape(5, 4)
grad = aa.backward(np.ones_like(out))
print (grad)

with tf.GradientTape() as tape:
    x, W = tf.constant(x), tf.constant(W)
    tape.watch(x)
    y = tf.matmul(x, W)
    loss = tf.reduce_sum(y)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf

x = np.random.normal(size=[5, 6])
aa = Relu()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(np.ones_like(out))
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.nn.relu(x)
    loss = tf.reduce_sum(y)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf
x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[1, 1]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1
print(label)
aa = Softmax()
out = aa.forward(x) # shape(5, 6)
grad = aa.backward(label)
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.nn.softmax(x)
    loss = tf.reduce_sum(y*label)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf

x = np.random.normal(size=[5, 6])
aa = Log()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(label)
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.math.log(x)
    loss = tf.reduce_sum(y*label)
    grads = tape.gradient(loss, x)
    print (grads)

(array([[ 1.20862557,  1.04914776,  2.12059047,  1.70653221, -0.37346108,
         0.50701596],
       [ 1.20862557,  1.04914776,  2.12059047,  1.70653221, -0.37346108,
         0.50701596],
       [ 1.20862557,  1.04914776,  2.12059047,  1.70653221, -0.37346108,
         0.50701596],
       [ 1.20862557,  1.04914776,  2.12059047,  1.70653221, -0.37346108,
         0.50701596],
       [ 1.20862557,  1.04914776,  2.12059047,  1.70653221, -0.37346108,
         0.50701596]]), array([[ 4.2514053 ,  4.2514053 ,  4.2514053 ,  4.2514053 ],
       [-3.82727536, -3.82727536, -3.82727536, -3.82727536],
       [-1.23688105, -1.23688105, -1.23688105, -1.23688105],
       [ 0.10782239,  0.10782239,  0.10782239,  0.10782239],
       [ 3.28468441,  3.28468441,  3.28468441,  3.28468441],
       [ 0.01227909,  0.01227909,  0.01227909,  0.01227909]]))


I0000 00:00:1742724777.529662  812299 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 74450 MB memory:  -> device: 0, name: NVIDIA A800 80GB PCIe, pci bus id: 0000:4f:00.0, compute capability: 8.0
I0000 00:00:1742724777.531526  812299 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79069 MB memory:  -> device: 1, name: NVIDIA A800 80GB PCIe, pci bus id: 0000:52:00.0, compute capability: 8.0
I0000 00:00:1742724777.532861  812299 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 79069 MB memory:  -> device: 2, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:56:00.0, compute capability: 8.0
I0000 00:00:1742724777.534271  812299 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 1488 MB memory:  -> device: 3, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:57:00.0, compute capability: 8.0
I0000 00:00:1742724777.536063  812299 gpu_device.cc:2022] Created

tf.Tensor(
[[ 1.20862557  1.04914776  2.12059047  1.70653221 -0.37346108  0.50701596]
 [ 1.20862557  1.04914776  2.12059047  1.70653221 -0.37346108  0.50701596]
 [ 1.20862557  1.04914776  2.12059047  1.70653221 -0.37346108  0.50701596]
 [ 1.20862557  1.04914776  2.12059047  1.70653221 -0.37346108  0.50701596]
 [ 1.20862557  1.04914776  2.12059047  1.70653221 -0.37346108  0.50701596]], shape=(5, 6), dtype=float64)
[[1. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1.]]
tf.Tensor(
[[1. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1.]], shape=(5, 6), dtype=float64)
[[0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]]
[[-6.86361544e-04  1.18361798e-01 -3.39530210e-02 -4.85195976e-05
  -7.03532713e-02 -1.33206247e-02]
 [ 1.72267403e-01  1.09022185e-05 -2.38830579e-04 -1.66578515e-01
  -5.45952789e-03 -1.43170379e-06]
 [-3.41245639e-09 -1.7

/tmp/ipykernel_812299/1473917.py:93: RuntimeWarning: invalid value encountered in log
  out = np.log(x+self.epsilon)


tf.Tensor(
[[-0.         -0.89018871  0.         -0.         -0.          0.        ]
 [-3.46647891 -1.863838    0.         -0.         -0.          0.        ]
 [-0.          0.          0.          1.1348597   0.          0.        ]
 [-0.         -0.         -0.         -0.         -0.         -3.7884822 ]
 [-0.47581375  0.          0.          0.          0.          0.        ]], shape=(5, 6), dtype=float64)


# Final Gradient Check


In [5]:
import tensorflow as tf

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(h2_log_grad)
print('--'*20)
# print(W2_grad)

with tf.GradientTape() as tape:
    x, W1, W2, label = tf.constant(x), tf.constant(W1), tf.constant(W2), tf.constant(label)
    tape.watch(W1)
    tape.watch(W2)
    h1 = tf.matmul(x, W1)
    h1_relu = tf.nn.relu(h1)
    h2 = tf.matmul(h1_relu, W2)
    prob = tf.nn.softmax(h2)
    log_prob = tf.math.log(prob)
    loss = tf.reduce_sum(label * log_prob)
    grads = tape.gradient(loss, [prob])
    print (grads[0].numpy())

[[0.00000000e+00 2.81892579e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.21596225e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.23774703e+01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.40881196e+03]
 [1.00057368e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
----------------------------------------
[[0.00000000e+00 2.81892579e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.21596225e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.23774703e+01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.40881197e+03]
 [1.00057368e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


## 建立模型


In [6]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss


In [7]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练


In [8]:
train_data, test_data = mnist_dataset()
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(50):
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
epoch 0 : loss 23.38736426959661 ; accuracy 0.10425
epoch 1 : loss 20.94965947086233 ; accuracy 0.19418333333333335
epoch 2 : loss 19.58886568650089 ; accuracy 0.2388
epoch 3 : loss 18.37613816364587 ; accuracy 0.28453333333333336
epoch 4 : loss 17.539773633314955 ; accuracy 0.31601666666666667
epoch 5 : loss 16.754379792575058 ; accuracy 0.34268333333333334
epoch 6 : loss 16.220649340314147 ; accuracy 0.3566666666666667
epoch 7 : loss 14.725425067090992 ; accuracy 0.4051166666666667
epoch 8 : loss 14.085312656327996 ; accuracy 0.43466666666666665
epoch 9 : loss 13.514467449953775 ; accuracy 0.4507
epoch 10 : loss 13.029359462603676 ; accuracy 0.4748
epoch 11 : loss 12.612513398161681 ; accuracy 0.48541666666666666
epoch 12 : loss 12.300744957802339 ; accuracy 0.5022333333333333
epoch 13 : loss 11.980427239095635 ; accuracy 0.50885
epoch 14 : loss 11.885233080347074 ; accuracy 0.5185833333333333
epoch 15 : loss 11.591815995545787 ; acc